# GB Network Visualization

In [95]:
import pandas as pd
import folium

Buses_combined = pd.read_csv('./buses_mapping/buses_combined.csv',dtype = {'bus385_id': str})


df_buses_501 = pd.read_csv('./buses_mapping/buses_501.csv')
df_buses_385 = pd.read_csv('./buses_mapping/buses_385.csv')
Buses_combined


,bus501_id,x_501,y_501,bus385_id,x_385,y_385,assigned_bus385_ids,assigned_x_385,assigned_y_385
0,way/975637991,-1.271945,60.293401,NaN,NaN,NaN,NaN,NaN,NaN
1,relation/8087295-275,-3.753620,58.575786,6441,-3.812256,58.546761,['6441'],[-3.812256],[58.546761]
2,way/495624575-275,-3.512715,58.564233,6442,-3.453827,58.566820,['6442'],[-3.453827],[58.56682]
3,way/567187951-275,-3.879158,58.512154,NaN,NaN,NaN,NaN,NaN,NaN
4,way/1262971878,-3.452185,58.479968,6443,-3.453827,58.490823,['6443'],[-3.453827],[58.490823]
...,...,...,...,...,...,...,...,...,...
496,way/49922626-400,-2.488599,50.623881,5274,-2.528229,50.628558,['5274'],[-2.528229],[50.628558]
497,way/26938722-400,-3.730387,50.471640,5210,-3.691406,50.477609,"['8016', '5210']","[-3.735352, -3.691406]","[50.498579, 50.477609]"
498,way/34656007-400,-4.242535,50.444878,5208,-4.244843,50.460127,['5208'],[-4.244843],[50.460127]
499,relation/9572254-400,-4.899645,50.395013,5207,-4.925995,50.368241,['5207'],[-4.925995],[50.368241]


In [96]:
import ast

x_501 = dict(zip(df_buses_501['bus_id'], df_buses_501['x']))
y_501 = dict(zip(df_buses_501['bus_id'], df_buses_501['y']))

bus_mapping = []

for _, row in Buses_combined.iterrows():
    bus501_id = row["bus501_id"]
    assigned_bus385_ids = row["assigned_bus385_ids"]
    
    if isinstance(assigned_bus385_ids, str):
        try:
            assigned_bus385_ids = ast.literal_eval(assigned_bus385_ids)
        except:
            assigned_bus385_ids = []

    if isinstance(assigned_bus385_ids, list):
        for b385_id in assigned_bus385_ids:
            bus_mapping.append({"bus385_id": b385_id, "bus501_id": bus501_id})

df_385_to_501 = pd.DataFrame(bus_mapping)
bus385_to_501 = dict(zip(df_385_to_501['bus385_id'], df_385_to_501['bus501_id']))


## Add 501 buses

In [97]:
map_GB = folium.Map(location=[54, -2], zoom_start=6, tiles="CartoDB positron")

for _, row in df_buses_501.iterrows():
    
    popup_html = f"""
    <b>Bus ID:</b> {row.get("bus_id", "")}<br>
    <b>x:</b> {row.get("x", "")}<br>
    <b>y:</b> {row.get("y", "")}
    """
    if row["bus_id"] in set(df_385_to_501['bus501_id']):
        folium.CircleMarker(
            location=[row["y"], row["x"]],
            radius=2,
            color="Green",
            fill=True,
            fill_opacity=0.7,
            popup=folium.Popup(popup_html, max_width=200)
        ).add_to(map_GB)
    else:
        folium.CircleMarker(
            location=[row["y"], row["x"]],
            radius=2,
            color="red",
            fill=True,
            fill_opacity=0.7,
            popup=folium.Popup(popup_html, max_width=200)
        ).add_to(map_GB)

## Add units

In [98]:
df_units_capacity = pd.read_csv('../GBPower_raw/bmus_prepared.csv')
df_units = pd.read_csv('../GBPower_raw/prepared_bmus.csv')

df_units['capacity'] = df_units_capacity['capacity']
df_units

,NationalGridBmUnit,carrier,bus,lon,lat,capacity
0,IBG-EGEN1,interconnector,4969,0.6530000000000001,51.46000000000001,0.0
1,IBG-ENGI1,interconnector,4969,0.6530000000000001,51.46000000000001,0.0
2,IBG-RSQ1,interconnector,4969,0.6530000000000001,51.46000000000001,0.0
3,IVD-VKL1,interconnector,7909,-0.221,52.931,0.0
4,IVG-VKL1,interconnector,7909,-0.221,52.931,0.0
...,...,...,...,...,...,...
1060,SFERD-1,demand_flex,distributed,distributed,distributed,240.0
1061,EAS-SEL01,demand_flex,distributed,distributed,distributed,0.0
1062,SEE-PGU01,demand_flex,distributed,distributed,distributed,0.0
1063,YEP-ATL01,demand_flex,distributed,distributed,distributed,0.0


In [99]:
for _, row in df_units.iterrows():
    if row.get("lon") == 'distributed':
        continue
    popup_html = f"""
    <b>Unit:</b> {row.get("NationalGridBmUnit", "")}<br>
    <b>x:</b> {row.get("lon", "")}<br>
    <b>y:</b> {row.get("lat", "")}<br>
    <b>Capacity:</b> {row.get("capacity", "")}
    """

    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=2,
        color="purple",
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(popup_html, max_width=200)
    ).add_to(map_GB)

## Add Lines between buses

In [100]:
df_lines = pd.read_csv('./buses_mapping/lines_501.csv')

x = dict(zip(df_buses_501['bus_id'], df_buses_501['x']))
y = dict(zip(df_buses_501['bus_id'], df_buses_501['y']))

In [103]:
for _, row in df_lines.iterrows():
    bus0, bus1 = row['bus0'], row['bus1']
    
    try:
        coord0 = (y[bus0], x[bus0])
        coord1 = (y[bus1], x[bus1])

        folium.PolyLine(
            locations=[coord0, coord1],
            color='blue',
            weight=1.2,
            opacity=0.8,
            popup=f"{bus0} → {bus1}"
        ).add_to(map_GB)
    
    except KeyError as e:
        print(f"Missing bus in coordinate mapping: {e}")
        continue

Missing bus in coordinate mapping: 'way/264275258-275'
Missing bus in coordinate mapping: 'way/264275258-275'


## Add Lines between buses and generators

In [104]:
for _, row in df_units.iterrows():
    gen = row['NationalGridBmUnit']
    bus385_id = row['bus']
    
    try:
        bus501_id = bus385_to_501[bus385_id]

        coord0 = (y_501[bus501_id], x_501[bus501_id])  
        coord1 = (row['lat'], row['lon'])              

        folium.PolyLine(
            locations=[coord0, coord1],
            color='black',
            weight=0.8,
            opacity=0.4,
            popup=f"{gen} → {bus385_id} → {bus501_id}"
        ).add_to(map_GB)

    except KeyError as e:
        print(f"missing: {e}")
        continue


missing: 'distributed'
missing: 'distributed'
missing: 'distributed'
missing: 'distributed'
missing: 'distributed'
missing: 'distributed'
missing: 'distributed'
missing: 'distributed'
missing: 'distributed'


## Add Links

In [105]:
df_links = pd.read_csv('links_GB.csv', sep = ',')
df_links

,link_id,bus0,bus1,voltage,p_nom,length,underground,under_construction,tags,geometry
0,relation/10377412-320-DC,way/753113423,way/636630368,320,1000,230137.18,t,f,relation/10377412,LINESTRING (-1.194439480272105 50.818035638860...
1,relation/13295785-515-DC,relation/12832053,way/642490160,515,1400,723362.06,t,f,relation/13295785,LINESTRING (-1.5404269162550226 55.14647596191...
2,relation/14126301-450-DC,way/109189896,way/920127890,450,1000,253393.42,t,f,relation/14126301,LINESTRING (0.7161575436002887 51.440498299145...
3,relation/15772117-320-DC,way/775577829,way/775577827,320,1000,51576.89,t,f,relation/15772117,LINESTRING (1.1448358124246543 51.098446880403...
4,relation/15775538-600-DC,way/297840585,way/375626500,600,2250,397643.50,t,f,relation/15775538,LINESTRING (-4.894821189854914 55.718036768358...
5,relation/15777152-320-DC,way/1262971878,way/975637991,320,600,282108.95,t,f,relation/15777152,LINESTRING (-3.452184943694071 58.479967903761...
6,relation/15781671-525-DC,way/1078532568,way/1101832649,525,1400,740934.57,t,f,relation/15781671,LINESTRING (-0.2365005345670103 52.92099253311...
7,relation/2127794-270-DC,way/23281217,way/1271288346,270,2000,69019.25,t,f,relation/2127794,LINESTRING (0.9751194497786296 51.105578116437...
8,relation/6914309-500-DC,way/137050527,way/148382435,500,500,61383.39,t,f,relation/6914309,LINESTRING (-5.770469013098239 54.843201485290...
9,relation/8099179-320-DC,way/1262971878,way/568527671,320,1200,161431.49,t,f,relation/8099179,LINESTRING (-3.452184943694071 58.479967903761...


In [106]:
Buses_EU = pd.read_csv('../Pypsa-eur_raw/buses.csv')
Buses_EU

,bus_id,voltage,dc,symbol,under_construction,tags,x,y,country,geometry
0,AL1-220,220,f,Substation,f,AL1,20.111702,42.096798,AL,POINT (20.111702399999995 42.09679778825765)
1,AL10-220,220,f,Substation,f,AL10,19.522149,40.478444,AL,POINT (19.522149214338864 40.47844442711058)
2,AL2-220,220,f,Substation,f,AL2,20.031168,42.069050,AL,POINT (20.031168199999993 42.06905038823308)
3,AL3-400,400,f,Substation,f,AL3,19.652495,42.010831,AL,POINT (19.652494799999996 42.01083068818158)
4,AL4-220,220,f,Substation,f,AL4,19.972850,41.584771,AL,POINT (19.9728498 41.58477058780021)
...,...,...,...,...,...,...,...,...,...,...
6732,way/99694910-380,380,f,Substation,f,way/99694910,7.789278,44.462732,IT,POINT (7.7892782005302115 44.46273179679329)
6733,way/99722046-225,225,f,Substation,f,way/99722046,-0.114171,47.206984,FR,POINT (-0.1141712110057602 47.206983706128305)
6734,way/99722046-400,400,f,Substation,f,way/99722046,-0.113930,47.207199,FR,POINT (-0.113930394824857 47.20719863601106)
6735,way/99826025-220,220,f,Substation,f,way/99826025,7.785444,44.875461,IT,POINT (7.785443587366926 44.87546070934813)


In [107]:
linked_buses = pd.Series(
    pd.concat([df_links['bus0'], df_links['bus1']]).unique()
)

Buses_linked = Buses_EU[Buses_EU['bus_id'].isin(linked_buses)]

### Added buses connected to the link

In [108]:
for _, row in Buses_linked.iterrows():
    popup_html = f"""
    <b>Bus_ID:</b> {row.get("bus_id", "")}<br>
    <b>x:</b> {row.get("x", "")}<br>
    <b>y:</b> {row.get("y", "")}<br>
    <b>Voltage:</b> {row.get("voltage", "")}
    """

    folium.CircleMarker(
        location=[row["y"], row["x"]],
        radius=2,
        color="orange",
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(popup_html, max_width=200)
    ).add_to(map_GB)

### Add links

In [109]:
x_linked = dict(zip(Buses_linked['bus_id'], Buses_linked['x']))
y_linked = dict(zip(Buses_linked['bus_id'], Buses_linked['y']))

for _, row in df_links.iterrows():
    bus0 = row['bus0']
    bus1 = row['bus1']
    
    try:
        coord0 = (y_linked[bus0], x_linked[bus0])  
        coord1 = (y_linked[bus1], x_linked[bus1])              

        folium.PolyLine(
            locations=[coord0, coord1],
            color='Yellow',
            weight=1,
            opacity=1,
            popup=f"{gen} → {bus385_id} → {bus501_id}"
        ).add_to(map_GB)

    except KeyError as e:
        print(f"missing: {e}")
        continue


## Add transformers

In [110]:
df_transformers = pd.read_csv('transformers_GB.csv', header = None)
df_transformers = df_transformers.drop(index=0).reset_index(drop=True)
df_transformers = df_transformers.iloc[:, :7]
df_transformers.columns = ['bus0', 'bus1', 'voltage_bus0', 'voltage_bus1', 's_nom', 'geometry_part1', 'geometry_part2']
df_transformers['geometry'] = df_transformers['geometry_part1'].astype(str) + ' ' + df_transformers['geometry_part2'].astype(str)
df_transformers = df_transformers.drop(columns=['geometry_part1', 'geometry_part2'])
df_transformers.insert(0, 'transformer_id', range(len(df_transformers)))

df_transformers = df_transformers.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_transformers

/tmp/ipykernel_304447/888247341.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_transformers = df_transformers.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,transformer_id,bus0,bus1,voltage_bus0,voltage_bus1,s_nom,geometry
0,0,GB74-275,GB74-400,275,400,3575,'LINESTRING (0.1651750409676874 51.44333737190...
1,1,relation/13110368-275,relation/13110368-400,275,400,7150,'LINESTRING (-1.255426653086503 52.86342638561...
2,2,relation/14921826-275,relation/14921826-400,275,400,3575,'LINESTRING (-2.9624053599211826 54.9421422587...
3,3,relation/7879831-275,relation/7879831-400,275,400,7150,'LINESTRING (-3.958613809406172 51.68587295880...
4,4,relation/7880111-275,relation/7880111-400,275,400,2766,'LINESTRING (-0.4842413102397478 51.3504784038...
...,...,...,...,...,...,...,...
74,74,way/94530230-275,way/94530230-400,275,400,5363,'LINESTRING (0.3898563459215954 51.45995983213...
75,75,way/95749833-275,way/95749833-400,275,400,7150,'LINESTRING (-3.300621406470803 51.63564619102...
76,76,way/96107758-275,way/96107758-400,275,400,7150,'LINESTRING (-1.9729298548661913 52.2508892310...
77,77,way/96905262-275,way/96905262-400,275,400,3575,'LINESTRING (-3.036220121280338 55.90844113772...


In [119]:
for _, row in df_transformers.iterrows():
    bus0 = row['bus0']
    bus1 = row['bus1']
    
    try:
        coord0 = (y[bus0], x[bus0])  
        coord1 = (y[bus1], x[bus1])              

        folium.PolyLine(
            locations=[coord0, coord1],
            color='brown',
            weight=1,
            opacity=1,
            popup=f"{gen} → {bus385_id} → {bus501_id}"
        ).add_to(map_GB)

    except KeyError as e:
        print(f"missing: {e}")
        continue

In [120]:
map_GB.save("GB_Network_Visualization.html")